In [1]:
import pandas as pd
import datetime
import numpy as np
import os.path
import wget
from zipfile import ZipFile
import shutil
import re
from multiprocessing import Pool
import multiprocessing as mp

In [2]:
## Variaveis globais

#pega o caminho do diretorio atual
diretorio = os.getcwd()
diretorio

#qual base de dados quero usar, "con" para consolidado e "ind" para individual
tipo = "con"

#Pega data corrente e ano corrente
date = datetime.date.today()
year = date.year

# ano inicial pra coleta dos dados
year_0 = 2010 
year_0_tri = year-2 # ano inicial pra coletoa dos dados trimestrais

In [3]:
# baixa todos os dados se não estiverem presentes, retorna uma lista de nomes de arquivos baixados
def baixa_dados(year_0, year, year_0_tri):
    # URL's onde estão os arquivos dfp e itr na CVM
    url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'
    url_quantidade = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/'
    url_ticker = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/'
    url_base_tri = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/'
    url_base_fca = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FCA/DADOS/"

    # Cria uma lista com os nomes dos arquivos de dados anuais (dfp) da CVM
    arquivos_zip = []
    for ano in range(year_0, year + 1):
        arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')

    # baixa os arquivos dfp
    arquivos_zip_dfp_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_dfp_baixados.append(arq)
        else:
            try:
                wget.download(url_base + arq, out=f"./Data/{arq}")
                arquivos_zip_dfp_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # Cria lista de arquivos ITR (trimestrais) pra baixar
    arquivos_zip = []
    for ano in range(year_0_tri, year + 1):
        arquivos_zip.append(f'itr_cia_aberta_{ano}.zip')

    # baixa os arquivos ITR (dados trimestrais) se não estiverem presentes
    arquivos_zip_itr_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_itr_baixados.append(arq)
        else:
            try:
                wget.download(url_base_tri + arq, out=f"./Data/{arq}")
                arquivos_zip_itr_baixados.append(arq)
            except:
                print(f"falha ao baixar {arq}")
                pass

    # baixa arquivo FCA

    # Cria lista de arquivos FCA  pra baixar. Este aqui só precisa baixar o do último ano disponível
    # Se não tiver do ano atual baixa o do ano passado
    arquivos_zip = []
    for ano in [year, year - 1]:
        arquivos_zip.append(f'fca_cia_aberta_{ano}.zip')

    arquivos_zip_fca_baixados = []
    for arq in arquivos_zip:
        if (os.path.isfile(f'./Data/{arq}')):
            print('%s já esta aqui' % arq)
            arquivos_zip_fca_baixados.append(arq)
            break
        else:
            try:
                wget.download(url_base_fca + arq, out=f"./Data/{arq}")
                arquivos_zip_fca_baixados.append(arq)
                break # se conseguiu baixar 1 já sai do loop
            except:
                print(f"falha ao baixar {arq}")

    return (arquivos_zip_dfp_baixados, arquivos_zip_itr_baixados, arquivos_zip_fca_baixados)

In [12]:
def processa_dados():
    print("Processando dados")
    # junta todos os anos de dados anuais em um arquivo só de cada tipo
    nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
             'DFC_MD_con', 'DFC_MI_con', 'DFC_MD_ind', 'DFC_MI_ind']
    for nome in nomes:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0, year):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/dfp_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em {nome}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/dfp_cia_aberta_{nome}_{year_0}-{year}.csv', index=False)
        print(f'DFP {nome} processado')

    # junta todos os anos de dados trimestrais em um arquivo só de cada tipo
    nomes_tri = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DRE_con', 'DRE_ind',
                 "DFC_MD_con", "DFC_MI_con", "DFC_MD_ind", "DFC_MI_ind"]
    for nome in nomes_tri:
        arquivo_bp = pd.DataFrame()
        for ano in range(year_0_tri, year + 1):
            try:
                arquivo_bp = pd.concat([arquivo_bp,
                                        pd.read_csv(f'./Data/temp/itr_cia_aberta_{nome}_{ano}.csv', sep=';',
                                                    decimal=',', encoding='ISO-8859-1')])
            except:
                print(f"falha em itr_cia_aberta_{nome}_{ano}")
                pass
        arquivo_bp.to_csv(f'./Data/processados/itr_cia_aberta_{nome}_{year_0_tri}-{year}.csv', index=False)
        print(f'ITR {nome} processado')

In [4]:
def extrai_arquivo(arq):
    try:
        ZipFile(f"./Data/{arq}", 'r').extractall('./Data/temp')
        print(f"{arq} extraido")
    except:
        print(f"erro ao extrair {arq}")

In [5]:
# extrai paralelo. não roda no jupyter

def extrai_paralelo(arquivos_para_extrair):

    #  descobre quantas threads tem o CPU atual e cria uma pool para extrair os dados com paralelismo
    procs = mp.cpu_count()
    pool = Pool(procs)

    # cria uma lista de pools para depois dar get
    lista_pool = []
    for arq in arquivos_para_extrair:
        lista_pool.append(pool.apply_async(extrai_arquivo, args=(arq,)))
    print("Extraindo")
    output = [item.get() for item in lista_pool]
    return(output)


In [6]:
# faz o usuario escolher uma empresa
def escolhe_empresa():
    lista_empresas = \
    pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_{tipo}_{year_0}-{year}.csv", header=0, sep=",")[
        ["DENOM_CIA", 'CD_CVM']].drop_duplicates()
    while True:
        busca = input("Entre com o nome da empresa: ")
        empresa = lista_empresas[lista_empresas['DENOM_CIA'].str.contains(busca, case=False)]
        print(empresa)
        ok = input("A empresa desejada está na primeira linha? (y/n) ")
        if ok == "y":
            return(busca)

In [7]:
arquivos_dfp, arquivos_itr, arquivos_fca = baixa_dados(year_0, year, year_0_tri)
arquivos_dfp, arquivos_itr, arquivos_fca

In [8]:
# extrai arquivos
arquivos_para_extrair = arquivos_dfp + arquivos_itr + arquivos_fca

#deleta pasta temporaria
try:
    shutil.rmtree(f'{diretorio}/Data/temp')
except:
    pass

# extrai sequencial
for arq in arquivos_para_extrair:
    extrai_arquivo(arq)

# extrai paralelo, não funciona no jupyter
#extrai_paralelo(arquivos_para_extrair)

dfp_cia_aberta_2010.zip extraido
dfp_cia_aberta_2011.zip extraido
dfp_cia_aberta_2012.zip extraido
dfp_cia_aberta_2013.zip extraido
dfp_cia_aberta_2014.zip extraido
dfp_cia_aberta_2015.zip extraido
dfp_cia_aberta_2016.zip extraido
dfp_cia_aberta_2017.zip extraido
dfp_cia_aberta_2018.zip extraido
dfp_cia_aberta_2019.zip extraido
dfp_cia_aberta_2020.zip extraido
dfp_cia_aberta_2021.zip extraido
dfp_cia_aberta_2022.zip extraido
itr_cia_aberta_2020.zip extraido
itr_cia_aberta_2021.zip extraido
itr_cia_aberta_2022.zip extraido
fca_cia_aberta_2022.zip extraido


,DENOM_CIA,CD_CVM
CNPJ_CIA,,
00.000.000/0001-91,BCO BRASIL S.A.,1023
00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206
00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437
00.070.698/0001-11,CIA ENERGETICA DE BRASILIA,14451
00.095.147/0001-02,VBC ENERGIA SA,14540
...,...,...
42.500.384/0001-51,OMEGA ENERGIA S.A.,26441
42.737.954/0001-21,"INTER & CO, INC.",57886
68.400.225/0001-79,CLARANET TECHNOLOGY S.A.,26549


In [10]:
# pega o nome do arquivo fca
arq_fca = [item for item in os.listdir("./Data/temp") if item.startswith("fca_cia_aberta_valor_mobiliario")]
arq_fca = arq_fca[0]

# copia o arquivo fca pra pasta processados
shutil.copyfile(f"./Data/temp/{arq_fca}", f"./Data/processados/{arq_fca}")

'./Data/processados/fca_cia_aberta_valor_mobiliario_2022.csv'

In [76]:
#cria lista de empresas
lista_empresas = pd.read_csv(f"./Data/processados/dfp_cia_aberta_BPA_{tipo}_{year_0}-{year}.csv", header=0, sep=",", )[["DENOM_CIA", 'CD_CVM', "CNPJ_CIA"]].drop_duplicates()
lista_empresas.set_index("CNPJ_CIA", inplace=True)

# coloca tickers do arquivo fca
lista_empresas = lista_empresas.join(pd.read_csv(f"./Data/processados/{arq_fca}", header=0, sep=";", encoding="latin1").set_index("CNPJ_Companhia")["Codigo_Negociacao"].dropna())

# pega tickers de outra fonte tbm pq na CVM é incompleto. 
# extraí de https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm
b3 = pd.read_csv("./Data/empresas_listadas_b3.csv", encoding="latin1", sep=";")[["Razão Social","Código"]]

# Cria a coluna CNPJ com os dados do index
lista_empresas = lista_empresas.rename_axis('CNPJ').reset_index()

# Junta os dados da b3 no arquivo original
lista_empresas = lista_empresas.merge(b3, left_on="DENOM_CIA", right_on="Razão Social", how="left").drop(columns=["Razão Social"])

#lista_empresas.to_excel("lista empresas.xlsx")
lista_empresas

,CNPJ,DENOM_CIA,CD_CVM,Codigo_Negociacao,Código
0,00.000.000/0001-91,BCO BRASIL S.A.,1023,BBAS3,BBAS
1,00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,BSLI3,BSLI
2,00.000.208/0001-00,BRB BCO DE BRASILIA S.A.,14206,BSLI4,BSLI
3,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,ELET3,ELET
4,00.001.180/0001-26,CENTRAIS ELET BRAS S.A. - ELETROBRAS,2437,ELET6,ELET
...,...,...,...,...,...
801,96.418.264/0218-02,LOJAS QUERO QUERO S.A.,25038,LJQQ3,NaN
802,97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,ODER3,ODER
803,97.191.902/0001-94,CONSERVAS ODERICH S.A.,4693,ODER4,ODER
804,97.837.181/0001-47,DURATEX S.A.,21091,DXCO3,NaN


In [ ]:
# agora tem que arrumar os dados. Acho melhor fazer um regex